# ⚙️ Advanced RAG Techniques & Optimization

> **Educational Notebook 24**: Deep dive into advanced RAG techniques, performance optimization, and system tuning

---

## 📋 Learning Objectives

By the end of this notebook, you will:

1. Understand advanced RAG techniques like Self-RAG, CRAG, and ReAct
2. Learn optimization strategies for performance and cost
3. Explore query routing and adaptive retrieval
4. Implement advanced evaluation techniques
5. Master system tuning for different use cases

## 🎯 Advanced RAG Paradigms

Beyond basic RAG, there are several advanced paradigms that can significantly improve performance:

- **Self-RAG**: Includes reasoning about retrieved content
- **CRAG**: Corrective RAG that validates and corrects retrieved information
- **ReAct**: Reasoning and Acting with thought chains
- **Adaptive RAG**: Dynamically adjusts retrieval strategy based on query complexity

Let's explore these concepts in detail!

## 🛠️ Setting Up Our Environment

First, let's set up our environment and import the necessary libraries:

In [1]:
import os
import sys
from pathlib import Path
import asyncio
import time
import random
from typing import List, Dict, Any, Optional, Callable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from enum import Enum
from functools import wraps

# Add project root to path
current = Path().resolve()
repo_root = None
for parent in [current, *current.parents]:
    if (parent / "src").exists() and (parent / "notebooks").exists():
        repo_root = parent
        break

if repo_root is None:
    raise RuntimeError("Could not locate rag-engine-mini root for imports")

sys.path.insert(0, str(repo_root))

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"Repository root: {repo_root}")
print("Environment setup complete!")

## 🧠 Advanced RAG Techniques

Let's explore the different advanced RAG techniques:

In [2]:
# Define RAG techniques
rag_techniques = {
    "Basic RAG": {
        "description": "Standard RAG pipeline: retrieve → augment → generate",
        "advantages": ["Simple to implement", "Good baseline performance"],
        "limitations": ["No validation of retrieved content", "Static retrieval strategy"],
        "use_case": "General purpose QA with well-structured documents"
    },
    "Self-RAG": {
        "description": "Includes reasoning about retrieved content with special tokens",
        "advantages": ["Validates retrieved information", "Can decide to not retrieve", "Self-reflection"],
        "limitations": ["Higher computational cost", "Requires special training"],
        "use_case": "Fact-checking applications, critical domains"
    },
    "CRAG (Corrective RAG)": {
        "description": "Uses knowledge graphs to validate and correct retrieved content",
        "advantages": ["Corrects factual errors", "Improves reliability"],
        "limitations": ["Requires knowledge graphs", "Complex implementation"],
        "use_case": "Medical, legal, scientific domains"
    },
    "ReAct (Reasoning + Acting)": {
        "description": "Combines reasoning steps with actions (like retrieval)",
        "advantages": ["Transparent reasoning", "Modular approach"],
        "limitations": ["More complex prompting", "Longer response times"],
        "use_case": "Multi-step reasoning tasks"
    },
    "Adaptive RAG": {
        "description": "Dynamically selects retrieval strategy based on query complexity",
        "advantages": ["Optimizes performance per query", "Balances cost and quality"],
        "limitations": ["Requires routing classifier", "More complex architecture"],
        "use_case": "Mixed query workloads"
    },
    "Active RAG": {
        "description": "Interactively refines retrieval based on intermediate results",
        "advantages": ["Iteratively improves results", "Handles ambiguous queries"],
        "limitations": ["Higher latency", "Risk of infinite loops"],
        "use_case": "Research assistance, complex queries"
    }
}

# Display the techniques
print("Advanced RAG Techniques:")
print("=" * 70)
for name, details in rag_techniques.items():
    print(f"\n{name}:")
    print(f"  Description: {details['description']}")
    print(f"  Advantages: {', '.join(details['advantages'])}")
    print(f"  Limitations: {', '.join(details['limitations'])}")
    print(f"  Use Case: {details['use_case']}")

## 🔄 Implementing Adaptive RAG

Let's implement a simplified version of Adaptive RAG that dynamically chooses retrieval strategies:

In [3]:
# Define query complexity classifier
class QueryComplexityClassifier:
    """
    Classifies query complexity to determine the appropriate retrieval strategy
    """
    
    def __init__(self):
        self.simple_keywords = [
            "what is", "define", "who is", "when", "where", "list", "name"
        ]
        
        self.complex_keywords = [
            "compare", "contrast", "analyze", "evaluate", "how does", "why", 
            "relationship", "impact", "effect", "should", "could", "would"
        ]
    
    def classify_complexity(self, query: str) -> str:
        """
        Classify query complexity as 'simple', 'medium', or 'complex'
        """
        query_lower = query.lower()
        
        # Count complex vs simple indicators
        simple_count = sum(1 for kw in self.simple_keywords if kw in query_lower)
        complex_count = sum(1 for kw in self.complex_keywords if kw in query_lower)
        
        # Consider query length as well
        length_score = min(len(query.split()), 20) / 20  # Normalize to 0-1
        
        if complex_count > simple_count:
            if length_score > 0.5:
                return "complex"
            else:
                return "medium"
        elif simple_count > 0:
            return "simple"
        else:
            # If no clear indicators, use length as heuristic
            if length_score < 0.3:
                return "simple"
            elif length_score < 0.6:
                return "medium"
            else:
                return "complex"

# Define retrieval strategies
class RetrievalStrategy(Enum):
    SIMPLE = "simple"
    HYBRID = "hybrid"
    RERANK = "rerank"
    KNOWLEDGE_GRAPH = "knowledge_graph"

# Adaptive RAG implementation
class AdaptiveRAG:
    """
    Adaptive RAG system that selects retrieval strategy based on query complexity
    """
    
    def __init__(self):
        self.classifier = QueryComplexityClassifier()
        
        # Map complexity to strategy
        self.complexity_to_strategy = {
            "simple": RetrievalStrategy.SIMPLE,
            "medium": RetrievalStrategy.HYBRID,
            "complex": RetrievalStrategy.RERANK
        }
    
    def select_strategy(self, query: str) -> RetrievalStrategy:
        """
        Select retrieval strategy based on query complexity
        """
        complexity = self.classifier.classify_complexity(query)
        strategy = self.complexity_to_strategy.get(complexity, RetrievalStrategy.HYBRID)
        
        print(f"Query: '{query[:50]}...' | Complexity: {complexity} | Strategy: {strategy.value}")
        return strategy
    
    def retrieve(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """
        Retrieve documents using the selected strategy
        """
        strategy = self.select_strategy(query)
        
        # Simulate retrieval with different strategies
        if strategy == RetrievalStrategy.SIMPLE:
            # Simple vector search - fastest
            time.sleep(0.1)  # Simulate API call
            retrieval_time = 0.1
        elif strategy == RetrievalStrategy.HYBRID:
            # Hybrid search - moderate speed
            time.sleep(0.3)  # Simulate API calls
            retrieval_time = 0.3
        elif strategy == RetrievalStrategy.RERANK:
            # Hybrid + rerank - slower but more accurate
            time.sleep(0.6)  # Simulate API calls + rerank
            retrieval_time = 0.6
        else:
            # Default to hybrid
            time.sleep(0.3)
            retrieval_time = 0.3
        
        # Generate mock results
        results = []
        for i in range(k):
            results.append({
                "id": f"doc_{random.randint(1000, 9999)}",
                "text": f"Relevant content for query '{query[:20]}' - chunk {i+1}",
                "score": round(random.uniform(0.5, 0.95), 3),
                "retrieval_time": retrieval_time
            })
        
        return results

# Test the adaptive RAG system
adaptive_rag = AdaptiveRAG()

test_queries = [
    "What is machine learning?",
    "How does neural networks work?",
    "Compare the performance of different neural network architectures for NLP tasks",
    "What are the ethical implications of AI on employment?",
    "List the types of algorithms.",
    "Analyze the impact of transformer models on natural language understanding benchmarks"
]

print("Adaptive RAG Strategy Selection:")
print("=" * 60)
all_results = []
for query in test_queries:
    results = adaptive_rag.retrieve(query, k=3)
    all_results.append((query, results))
    
    # Show first result for each query
    print(f"  Top result score: {results[0]['score']}")
    print()

## 🚀 Performance Optimization Strategies

Let's explore various optimization strategies for RAG systems:

In [4]:
# Define optimization strategies
optimization_strategies = {
    "Caching": {
        "description": "Cache embeddings, retrieved chunks, and LLM responses",
        "implementation": ["Redis for embeddings", "In-memory for recent queries", "Persistent for common queries"],
        "impact": "Significant reduction in latency and cost"
    },
    "Compression": {
        "description": "Reduce embedding dimensionality to improve speed",
        "implementation": ["PCA", "Autoencoders", "Quantization"],
        "impact": "Faster retrieval at slight accuracy cost"
    },
    "Early Exit": {
        "description": "Stop retrieval when confidence threshold is met",
        "implementation": ["Confidence scoring", "Threshold setting"],
        "impact": "Reduced latency for confident answers"
    },
    "Query Rewriting": {
        "description": "Improve query formulation before retrieval",
        "implementation": ["Query expansion", "Query reformulation", "Query decomposition"],
        "impact": "Better retrieval accuracy"
    },
    "Hybrid Search": {
        "description": "Combine semantic and keyword search for better results",
        "implementation": ["RRF fusion", "Weighted combination", "Cross-encoder reranking"],
        "impact": "Improved recall and precision"
    },
    "Multi-Tier Retrieval": {
        "description": "Use different retrieval methods at different stages",
        "implementation": ["Coarse-to-fine", "Filtering approaches"],
        "impact": "Better performance for large collections"
    }
}

# Display the strategies
print("RAG Performance Optimization Strategies:")
print("=" * 80)
for name, details in optimization_strategies.items():
    print(f"\n{name}:")
    print(f"  Description: {details['description']}")
    print(f"  Implementation: {', '.join(details['implementation'])}")
    print(f"  Impact: {details['impact']}")

## 📊 Implementing a Caching Layer

Let's implement a multi-level caching system for RAG optimization:

In [5]:
# Implement a multi-level cache
class MultiLevelCache:
    """
    Multi-level cache for RAG optimization
    Level 1: In-memory cache (fastest, smallest)
    Level 2: Redis-like cache (medium speed, larger)
    Level 3: Persistent cache (slowest, largest)
    """
    
    def __init__(self, l1_size: int = 100, l2_size: int = 1000, l3_size: int = 10000):
        self.l1_cache = {}  # In-memory cache
        self.l1_access_times = {}  # Track access times
        self.l1_size = l1_size
        
        self.l2_cache = {}  # Simulated Redis cache
        self.l2_access_times = {}
        self.l2_size = l2_size
        
        self.l3_cache = {}  # Simulated persistent cache
        self.l3_access_times = {}
        self.l3_size = l3_size
        
        # Stats
        self.hits = {"l1": 0, "l2": 0, "l3": 0}
        self.misses = 0
        
    def _evict_lru(self, cache_dict, access_times, max_size):
        """Evict least recently used items if cache is full"""
        if len(cache_dict) >= max_size:
            # Find oldest access time
            oldest_key = min(access_times.keys(), key=lambda k: access_times[k])
            del cache_dict[oldest_key]
            del access_times[oldest_key]
    
    def get(self, key: str):
        """Get value from cache hierarchy"""
        current_time = time.time()
        
        # Check L1 first
        if key in self.l1_cache:
            self.l1_access_times[key] = current_time
            self.hits["l1"] += 1
            return self.l1_cache[key], "l1"
        
        # Check L2
        if key in self.l2_cache:
            # Move to L1 (promote)
            value = self.l2_cache[key]
            self._evict_lru(self.l1_cache, self.l1_access_times, self.l1_size)
            self.l1_cache[key] = value
            self.l1_access_times[key] = current_time
            
            # Remove from L2
            del self.l2_cache[key]
            del self.l2_access_times[key]
            
            self.hits["l2"] += 1
            return value, "l2"
        
        # Check L3
        if key in self.l3_cache:
            # Move to L1 (promote)
            value = self.l3_cache[key]
            self._evict_lru(self.l1_cache, self.l1_access_times, self.l1_size)
            self.l1_cache[key] = value
            self.l1_access_times[key] = current_time
            
            # Remove from L3
            del self.l3_cache[key]
            del self.l3_access_times[key]
            
            self.hits["l3"] += 1
            return value, "l3"
        
        # Miss
        self.misses += 1
        return None, "miss"
    
    def set(self, key: str, value: Any, level: str = "auto"):
        """Set value in cache hierarchy"""
        current_time = time.time()
        
        if level == "auto":
            # Default to L1 for new items
            level = "l1"
        
        if level == "l1":
            self._evict_lru(self.l1_cache, self.l1_access_times, self.l1_size)
            self.l1_cache[key] = value
            self.l1_access_times[key] = current_time
        elif level == "l2":
            self._evict_lru(self.l2_cache, self.l2_access_times, self.l2_size)
            self.l2_cache[key] = value
            self.l2_access_times[key] = current_time
        elif level == "l3":
            self._evict_lru(self.l3_cache, self.l3_access_times, self.l3_size)
            self.l3_cache[key] = value
            self.l3_access_times[key] = current_time
    
    def get_stats(self):
        """Get cache statistics"""
        total_requests = sum(self.hits.values()) + self.misses
        hit_rate = sum(self.hits.values()) / total_requests if total_requests > 0 else 0
        
        return {
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": hit_rate,
            "total_requests": total_requests,
            "l1_size": len(self.l1_cache),
            "l2_size": len(self.l2_cache),
            "l3_size": len(self.l3_cache)
        }

# Test the multi-level cache
cache = MultiLevelCache(l1_size=5, l2_size=10, l3_size=20)

# Simulate some cache operations
test_keys = [f"query_{i}" for i in range(15)]

print("Testing Multi-Level Cache:")
print("=" * 40)

for i, key in enumerate(test_keys):
    # Every other key is a repeat (should hit)
    if i > 0 and random.random() < 0.3:  # 30% chance of repeat
        key = random.choice(test_keys[:i])
    
    value, level = cache.get(key)
    
    if level == "miss":
        # Simulate expensive computation
        value = f"Computed result for {key} at {time.time()}"
        cache.set(key, value)
        print(f"Miss: {key} - computed and stored")
    else:
        print(f"Hit: {key} - retrieved from {level}")

# Show stats
stats = cache.get_stats()
print(f"\nCache Statistics:")
print(f"  Total requests: {stats['total_requests']}")
print(f"  Hit rate: {stats['hit_rate']:.2%}")
print(f"  Hits breakdown: {stats['hits']}")
print(f"  Misses: {stats['misses']}")
print(f"  Cache sizes: L1={stats['l1_size']}, L2={stats['l2_size']}, L3={stats['l3_size']}")

## 📈 Query Routing and Dynamic Configuration

Let's implement a system for routing queries to different retrieval configurations:

In [6]:
# Define different retrieval configurations
@dataclass
class RetrievalConfig:
    name: str
    strategy: str  # "semantic", "keyword", "hybrid", "graph"
    k: int  # number of documents to retrieve
    use_rerank: bool
    use_query_expansion: bool
    chunk_size: int  # in tokens
    performance_cost: float  # arbitrary cost metric

# Define query router
class QueryRouter:
    """
    Routes queries to optimal retrieval configurations
    """
    
    def __init__(self):
        # Define available configurations
        self.configs = [
            RetrievalConfig(
                name="fast_basic",
                strategy="semantic",
                k=3,
                use_rerank=False,
                use_query_expansion=False,
                chunk_size=256,
                performance_cost=0.3
            ),
            RetrievalConfig(
                name="balanced_hybrid",
                strategy="hybrid",
                k=5,
                use_rerank=False,
                use_query_expansion=True,
                chunk_size=512,
                performance_cost=0.6
            ),
            RetrievalConfig(
                name="accurate_rerank",
                strategy="hybrid",
                k=10,
                use_rerank=True,
                use_query_expansion=True,
                chunk_size=512,
                performance_cost=0.9
            ),
            RetrievalConfig(
                name="graph_enhanced",
                strategy="graph",
                k=7,
                use_rerank=False,
                use_query_expansion=True,
                chunk_size=512,
                performance_cost=0.8
            )
        ]
        
        # Define routing rules
        self.routing_rules = [
            {"condition": lambda q: len(q.split()) <= 3, "config": "fast_basic", "priority": 1},
            {"condition": lambda q: any(word in q.lower() for word in ["compare", "analyze", "evaluate"]), 
             "config": "accurate_rerank", "priority": 1},
            {"condition": lambda q: any(word in q.lower() for word in ["relationship", "connection", "linked to"]), 
             "config": "graph_enhanced", "priority": 1},
            {"condition": lambda q: True, "config": "balanced_hybrid", "priority": 0}  # default
        ]
    
    def route_query(self, query: str) -> RetrievalConfig:
        """
        Route query to optimal configuration
        """
        for rule in sorted(self.routing_rules, key=lambda r: r["priority"], reverse=True):
            if rule["condition"](query):
                config_name = rule["config"]
                config = next((c for c in self.configs if c.name == config_name), None)
                return config
        
        # Fallback to default
        return self.configs[1]  # balanced_hybrid
    
    def get_all_configs(self) -> List[RetrievalConfig]:
        """
        Get all available configurations
        """
        return self.configs

# Test the query router
router = QueryRouter()

test_queries = [
    "AI",  # Very short
    "What is RAG?",  # Short
    "Compare transformer and RNN architectures",  # Complex analytical
    "How are neural networks connected to deep learning?",  # Relationship query
    "Explain machine learning basics",  # General
    "What are the ethical implications of AI?"  # Complex
]

print("Query Routing System:")
print("=" * 50)

for query in test_queries:
    config = router.route_query(query)
    print(f"Query: '{query}'")
    print(f"  → Config: {config.name}")
    print(f"  → Strategy: {config.strategy}, k={config.k}")
    print(f"  → Rerank: {config.use_rerank}, Expansion: {config.use_query_expansion}")
    print(f"  → Cost: {config.performance_cost}\n")

# Show all configurations
print("Available Retrieval Configurations:")
print("=" * 50)
for config in router.get_all_configs():
    print(f"{config.name}: {config.strategy}, k={config.k}, rerank={config.use_rerank}, cost={config.performance_cost}")

## 🧪 Advanced Evaluation Techniques

Let's implement advanced evaluation techniques for RAG systems:

In [7]:
# Implement advanced evaluation metrics
class AdvancedRAGEvaluator:
    """
    Advanced evaluation techniques for RAG systems
    """
    
    def __init__(self):
        pass
    
    def evaluate_faithfulness(self, answer: str, context: str) -> float:
        """
        Evaluate how faithful the answer is to the provided context
        """
        # Simplified implementation - in practice, use LLM-based evaluation
        answer_words = set(answer.lower().split())
        context_words = set(context.lower().split())
        
        if not answer_words:
            return 0.0
            
        overlap = answer_words.intersection(context_words)
        faithfulness = len(overlap) / len(answer_words)
        
        # Cap at 1.0
        return min(1.0, faithfulness)
    
    def evaluate_answer_relevance(self, question: str, answer: str) -> float:
        """
        Evaluate how relevant the answer is to the question
        """
        question_words = set(question.lower().split())
        answer_words = set(answer.lower().split())
        
        if not question_words:
            return 0.0
            
        overlap = question_words.intersection(answer_words)
        relevance = len(overlap) / len(question_words)
        
        return min(1.0, relevance)
    
    def evaluate_context_recall(self, context: str, answer: str) -> float:
        """
        Evaluate how much of the answer content is supported by the context
        """
        answer_words = set(answer.lower().split())
        context_words = set(context.lower().split())
        
        if not answer_words:
            return 0.0
            
        overlap = answer_words.intersection(context_words)
        recall = len(overlap) / len(answer_words)
        
        return min(1.0, recall)
    
    def evaluate_compression_ratio(self, original_context: str, retrieved_context: str) -> float:
        """
        Evaluate how well the retrieval system compresses relevant information
        """
        orig_len = len(original_context)
        ret_len = len(retrieved_context)
        
        if orig_len == 0:
            return 0.0
        
        # Compression ratio - lower is better (more compression)
        # But we want to balance compression with information retention
        compression = ret_len / orig_len
        
        # Invert so higher is better
        return 1 - min(compression, 1.0)
    
    def evaluate_latency_cost_tradeoff(self, latency: float, cost: float, quality: float) -> float:
        """
        Evaluate the tradeoff between latency, cost, and quality
        """
        # Normalize values (assuming reasonable ranges)
        norm_latency = min(latency / 5.0, 1.0)  # Assume 5s is very slow
        norm_cost = min(cost / 0.1, 1.0)  # Assume $0.1 is very expensive
        
        # Calculate a combined score
        # Penalize high latency and cost, reward high quality
        score = quality - (norm_latency * 0.2) - (norm_cost * 0.2)
        
        return max(0.0, score)

# Test the evaluator
evaluator = AdvancedRAGEvaluator()

# Sample data for evaluation
samples = [
    {
        "question": "What is machine learning?",
        "context": "Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to learn from data without being explicitly programmed. It includes supervised, unsupervised, and reinforcement learning approaches.",
        "answer": "Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to learn from data without being explicitly programmed.",
        "latency": 1.2,
        "cost": 0.02
    },
    {
        "question": "How does neural networks work?",
        "context": "Neural networks are computing systems vaguely inspired by the biological neural networks that constitute animal brains. They consist of interconnected nodes called neurons that form layers. Each connection has a weight that adjusts during training.",
        "answer": "Neural networks consist of interconnected nodes called neurons that form layers. Each connection has a weight that adjusts during training.",
        "latency": 2.5,
        "cost": 0.05
    }
]

print("Advanced RAG Evaluation:")
print("=" * 60)

for i, sample in enumerate(samples):
    print(f"Sample {i+1}:")
    print(f"  Question: {sample['question'][:50]}...")
    
    # Calculate metrics
    faithfulness = evaluator.evaluate_faithfulness(sample['answer'], sample['context'])
    relevance = evaluator.evaluate_answer_relevance(sample['question'], sample['answer'])
    recall = evaluator.evaluate_context_recall(sample['context'], sample['answer'])
    compression = evaluator.evaluate_compression_ratio(sample['context'], sample['context'][:200])
    tradeoff_score = evaluator.evaluate_latency_cost_tradeoff(
        sample['latency'], sample['cost'], (faithfulness + relevance + recall) / 3
    )
    
    print(f"  Faithfulness: {faithfulness:.3f}")
    print(f"  Answer Relevance: {relevance:.3f}")
    print(f"  Context Recall: {recall:.3f}")
    print(f"  Compression: {compression:.3f}")
    print(f"  Trade-off Score: {tradeoff_score:.3f}")
    print()

## 📊 Performance Benchmarking

Let's create a benchmarking system to compare different RAG configurations:

In [8]:
# Benchmark different configurations
class RAGBenchmark:
    """
    Benchmark different RAG configurations
    """
    
    def __init__(self):
        self.evaluator = AdvancedRAGEvaluator()
        
    def benchmark_config(self, config: RetrievalConfig, test_samples: List[Dict]) -> Dict[str, Any]:
        """
        Benchmark a specific configuration on test samples
        """
        results = []
        
        for sample in test_samples:
            # Simulate retrieval and generation based on config
            # This is a simplified simulation
            
            # Calculate latency based on config properties
            base_latency = 0.5
            strategy_factor = {"semantic": 1.0, "keyword": 1.2, "hybrid": 1.5, "graph": 1.8}[config.strategy]
            rerank_factor = 1.5 if config.use_rerank else 1.0
            expansion_factor = 1.1 if config.use_query_expansion else 1.0
            
            simulated_latency = base_latency * strategy_factor * rerank_factor * expansion_factor
            
            # Simulate quality based on config properties
            quality = 0.5  # base quality
            quality += 0.2 if config.use_rerank else 0.0
            quality += 0.1 if config.use_query_expansion else 0.0
            quality += 0.1 if config.k >= 5 else 0.0
            quality = min(quality, 1.0)
            
            # Calculate cost based on operations performed
            cost = 0.01 * config.k  # base cost per retrieved doc
            if config.use_rerank:
                cost += 0.005 * config.k
            if config.use_query_expansion:
                cost += 0.002
            
            # Run evaluation metrics
            faithfulness = self.evaluator.evaluate_faithfulness(sample['answer'], sample['context'])
            relevance = self.evaluator.evaluate_answer_relevance(sample['question'], sample['answer'])
            
            result = {
                "latency": simulated_latency,
                "quality": quality,
                "cost": cost,
                "faithfulness": faithfulness,
                "relevance": relevance,
                "overall_score": self.evaluator.evaluate_latency_cost_tradeoff(simulated_latency, cost, quality)
            }
            
            results.append(result)
        
        # Aggregate results
        agg_results = {
            "config_name": config.name,
            "avg_latency": np.mean([r["latency"] for r in results]),
            "avg_quality": np.mean([r["quality"] for r in results]),
            "avg_cost": np.mean([r["cost"] for r in results]),
            "avg_faithfulness": np.mean([r["faithfulness"] for r in results]),
            "avg_relevance": np.mean([r["relevance"] for r in results]),
            "avg_overall_score": np.mean([r["overall_score"] for r in results]),
            "throughput": len(results) / sum(r["latency"] for r in results),  # queries per second
            "cost_per_query": np.mean([r["cost"] for r in results])
        }
        
        return agg_results
    
    def run_comparison(self, configs: List[RetrievalConfig], test_samples: List[Dict]) -> pd.DataFrame:
        """
        Compare multiple configurations
        """
        results = []
        
        for config in configs:
            print(f"Benchmarking {config.name}...")
            result = self.benchmark_config(config, test_samples)
            results.append(result)
        
        return pd.DataFrame(results)

# Run benchmarks
benchmark = RAGBenchmark()
configs = router.get_all_configs()

# Extend our samples for benchmarking
extended_samples = samples * 5  # Repeat samples for better statistics

print("Running RAG Configuration Benchmarks:")
print("=" * 60)

benchmark_results = benchmark.run_comparison(configs, extended_samples)
print(benchmark_results.round(3))

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Latency
axes[0, 0].bar(benchmark_results['config_name'], benchmark_results['avg_latency'])
axes[0, 0].set_title('Avg. Latency by Config')
axes[0, 0].set_ylabel('Latency (s)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Quality
axes[0, 1].bar(benchmark_results['config_name'], benchmark_results['avg_quality'])
axes[0, 1].set_title('Avg. Quality by Config')
axes[0, 1].set_ylabel('Quality Score')
axes[0, 1].tick_params(axis='x', rotation=45)

# Cost
axes[0, 2].bar(benchmark_results['config_name'], benchmark_results['avg_cost'])
axes[0, 2].set_title('Avg. Cost by Config')
axes[0, 2].set_ylabel('Cost ($ per query)')
axes[0, 2].tick_params(axis='x', rotation=45)

# Faithfulness
axes[1, 0].bar(benchmark_results['config_name'], benchmark_results['avg_faithfulness'])
axes[1, 0].set_title('Avg. Faithfulness by Config')
axes[1, 0].set_ylabel('Faithfulness Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# Throughput
axes[1, 1].bar(benchmark_results['config_name'], benchmark_results['throughput'])
axes[1, 1].set_title('Throughput by Config')
axes[1, 1].set_ylabel('Queries/sec')
axes[1, 1].tick_params(axis='x', rotation=45)

# Overall Score
axes[1, 2].bar(benchmark_results['config_name'], benchmark_results['avg_overall_score'])
axes[1, 2].set_title('Overall Score by Config')
axes[1, 2].set_ylabel('Score')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🎛️ System Tuning Recommendations

Based on our analysis, let's provide recommendations for tuning the RAG system:

In [9]:
# Generate system tuning recommendations
def generate_tuning_recommendations(benchmark_df: pd.DataFrame) -> List[str]:
    """
    Generate tuning recommendations based on benchmark results
    """
    recommendations = []
    
    # Find best config for different scenarios
    best_latency_idx = benchmark_df['avg_latency'].idxmin()
    best_quality_idx = benchmark_df['avg_quality'].idxmax()
    best_cost_idx = benchmark_df['avg_cost'].idxmin()
    best_overall_idx = benchmark_df['avg_overall_score'].idxmax()
    
    best_latency_config = benchmark_df.loc[best_latency_idx, 'config_name']
    best_quality_config = benchmark_df.loc[best_quality_idx, 'config_name']
    best_cost_config = benchmark_df.loc[best_cost_idx, 'config_name']
    best_overall_config = benchmark_df.loc[best_overall_idx, 'config_name']
    
    recommendations.append(
        f"⚡ For lowest latency: Use {best_latency_config} configuration"
    )
    recommendations.append(
        f"🎯 For highest quality: Use {best_quality_config} configuration"
    )
    recommendations.append(
        f"💰 For lowest cost: Use {best_cost_config} configuration"
    )
    recommendations.append(
        f"🏆 For best overall score: Use {best_overall_config} configuration"
    )
    
    # Additional recommendations based on data
    if benchmark_df['avg_latency'].std() > 0.2:
        recommendations.append(
            "🔄 Consider implementing adaptive configuration switching based on query characteristics"
        )
    
    if benchmark_df['avg_cost'].mean() > 0.05:
        recommendations.append(
            "💸 Implement caching strategies to reduce API costs"
        )
    
    if benchmark_df['avg_faithfulness'].mean() < 0.7:
        recommendations.append(
            "🔍 Consider improving the retrieval precision to enhance faithfulness"
        )
    
    return recommendations

# Generate recommendations
recommendations = generate_tuning_recommendations(benchmark_results)

print("System Tuning Recommendations:")
print("=" * 50)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Create a summary of the advanced RAG concepts
print(f"\nAdvanced RAG Optimization Summary:")
print("=" * 50)
print("• Implemented Adaptive RAG with query complexity classification")
print("• Created multi-level caching system for performance")
print("• Developed query routing based on content and intent")
print("• Established advanced evaluation metrics (faithfulness, relevance)")
print("• Benchmarked configurations for performance-cost tradeoffs")
print("• Provided system tuning recommendations")

## 📚 Key Takeaways

Advanced RAG optimization involves multiple interconnected strategies:

1. **Adaptive Approaches**: Adjust retrieval strategy based on query characteristics
2. **Caching**: Implement multi-level caching to reduce latency and cost
3. **Query Routing**: Direct queries to optimal configurations
4. **Advanced Evaluation**: Look beyond simple metrics to assess quality
5. **Configuration Benchmarking**: Systematically compare different setups
6. **System Tuning**: Optimize based on specific requirements and constraints

The RAG Engine Mini project demonstrates many of these concepts with production-grade implementations including caching, hybrid search, and configurable retrieval strategies.

## 🏁 Conclusion

In this notebook, we've explored advanced RAG techniques and optimization strategies:

- We learned about advanced RAG paradigms like Self-RAG, CRAG, and Adaptive RAG
- We implemented an adaptive system that selects retrieval strategies based on query complexity
- We created a multi-level caching system for performance optimization
- We developed advanced evaluation metrics for assessing RAG quality
- We benchmarked different configurations to understand performance trade-offs
- We provided system tuning recommendations based on empirical data

These advanced techniques are essential for building production RAG systems that balance quality, performance, and cost. The RAG Engine Mini project provides a solid foundation for implementing these advanced techniques in real-world applications.

Continue to experiment with different configurations and optimization strategies to find the best approach for your specific use case!